In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
path = 'D:/Work/DAandML/kaggle.com/ForestCoverType'

In [3]:
train = pd.read_csv(path+'/train.csv')
test = pd.read_csv(path+'/test.csv')
train.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [4]:
r, c = train.shape

zero_list = []
for i in range(c):
    if(train.iloc[:,i].std()==0):
        zero_list.append(train.columns[i])

del_list = zero_list
del_list.append('Id')
del_list.append('Cover_Type')

feature_cols = [col for col in train.columns if col not in del_list]

In [5]:
x = train.loc[:,feature_cols].values
y = train.loc[:,'Cover_Type'].values
x_test = test.loc[:,feature_cols].values

In [8]:
#Import libraries for data transformations
from sklearn.preprocessing import Normalizer
from sklearn.cross_validation import train_test_split

In [9]:
# The categorical variables begin
size = 10

# Normalize the feature values (apply for non-categorical only)
x_norm_size = Normalizer().fit_transform(x[:,:size])
x_norm = np.concatenate((x_norm_size, x[:,size:]), axis=1)

x_test_norm_size = Normalizer().fit_transform(x_test[:,:size])
x_test_norm = np.concatenate((x_test_norm_size, x_test[:,size:]), axis=1)

C:\Users\anhnt109\Anaconda2\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, DataConversionWarning)
C:\Users\anhnt109\Anaconda2\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, DataConversionWarning)


In [10]:
# 4-rd try, SVC and Feature Engineering
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
clf = SVC()

parameter_grid = {'C': [1, 10, 100, 1000],
                  "gamma": [0.001, 0.0001],
                  'kernel': ['rbf', 'linear','poly']}

In [11]:
cross_validation = StratifiedKFold(y, n_folds=10)
grid_search = GridSearchCV(clf,
                           param_grid=parameter_grid, cv=cross_validation)

In [12]:
#grid_search.fit(x_norm_train, y_train)
grid_search.fit(x_norm, y)

GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[5 5 ..., 3 3], n_folds=10, shuffle=False, random_state=None),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'kernel': ['rbf', 'linear', 'poly'], 'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [16]:
grid_search.best_score_

0.64166666666666672

In [15]:
grid_search.best_estimator_

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [16]:
#dir()

In [17]:
clf_grid = grid_search.best_estimator_
#clf_grid.score(x_norm_val, y_val)

In [18]:
y_test = clf_grid.predict(x_test_norm)

In [19]:
result = test[['Id']]
result.loc[:,'Cover_Type'] = y_test
result.to_csv('SubmissionSVC.csv', index=False)

C:\Users\anhnt109\Anaconda2\lib\site-packages\pandas\core\indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\anhnt109\Anaconda2\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
